In [3]:
import sys
from g_python.gextension import Extension
from g_python.hmessage import Direction, HMessage
from g_python.hpacket import HPacket
from g_python import hparsers
from g_python import htools
import sys
from g_python.gextension import Extension
from g_python.hdirection import Direction
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Users\ciroj\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [89]:
extension_info = {
    "title": "Extension stuff",
    "description": "g_python test",
    "version": "1.0",
    "author": "sirjonasxx"
}
sys.argv = ['-p', ' 9092']
ext = Extension(extension_info, sys.argv)   # sys.argv são os argumentos da linha de comando, por exemplo ['-p', '9092'] (porta das extensões do G-Earth)
ext.start()

# Variável para armazenar as mensagens
captured_messages = []

# Função para verificar se uma string está codificada em UTF-8
def is_utf8(data):
    try:
        data.encode('utf-8')
        return True
    except UnicodeError:
        return False

# Função de interceptação e filtragem de mensagens
def all_packets(message):
    packet = message.packet
    packet_str = packet.g_string(ext)
    
    # Extrair a frase da mensagem removendo números e arrays e caracteres extras
    cleaned_message = re.sub(r'[\[\]\d+Æ!)]', '', packet_str).strip()
    
    # Verifica se a mensagem contém a palavra "Rasci" (case-insensitive)
    if re.search(r'rasci', cleaned_message, re.IGNORECASE):
        # Verifica se a mensagem está codificada em UTF-8
        if is_utf8(cleaned_message):
            # Armazena a mensagem capturada
            captured_messages.append(cleaned_message)
            print(f"Mensagem capturada: {cleaned_message}")

# Intercepta pacotes indo para o cliente e para o servidor
ext.intercept(Direction.TO_CLIENT, all_packets)

In [ ]:
ext.send_to_server(HPacket("Chat", "Isso é um teste, favor ignorar", 1, 1))

In [63]:
# intercept & print all packets
def all_packets(message):
    packet = message.packet
    print(packet.g_string(ext))

ext.intercept(Direction.TO_CLIENT, all_packets)
ext.intercept(Direction.TO_SERVER, all_packets)

In [72]:
# intercept & parse specific packets
def on_walk(message):
    (x, y) = message.packet.read('ii')
    print("Walking to x:{}, y={}".format(x, y))

ext.intercept(Direction.TO_SERVER, on_walk, 'RoomUserWalk')

In [51]:
def on_speech(message):
    (text, color, index) = message.packet.read('sii')
    message.is_blocked = (text == 'blocked')  # block packet if speech equals "blocked"
    print("User said: {}".format(text))

    ext.intercept(Direction.TO_SERVER, on_speech, 'RoomUserTalk')

In [49]:
def speech_out(msg):
    text, bubble, id = msg.packet.read('sii')
    ext.write_to_console(f"Message send -> message : '{text}', bubble : {bubble}")

ext.intercept(Direction.TO_SERVER, speech_out, 'Chat')

In [53]:

def speech_in(msg):
    index, text, _, bubble, _, id = msg.packet.read('isiiii')
    ext.write_to_console(f"Message receive -> index : {index}, message : {text}, bubble : {bubble}", color='blue', mention_title=False)
    ext.intercept(Direction.TO_CLIENT, speech_in, 'Chat')

In [28]:
def on_connection_start():
    print('Connected with: {}:{}'.format(ext.connection_info['host'], ext.connection_info['port']))
    print(ext.packet_infos)


ext.on_event('double_click', lambda: print('Extension has been clicked'))
ext.on_event('init', lambda: print('Initialized with g-earth'))
ext.on_event('connection_start', on_connection_start)
ext.on_event('connection_end', lambda: print('Connection ended'))

print(ext.request_flags())

[]


In [38]:
def all_packets(message):
    packet = message.packet
    s = packet.g_string(ext)
    expr = packet.g_expression(ext)
    print('{} --> {}'.format(message.direction.name, s))
    if expr != '':
        print(expr)
    print('------------------------------------')

ext.intercept(Direction.TO_CLIENT, all_packets)
ext.intercept(Direction.TO_SERVER, all_packets)

In [73]:
ext.send_to_server(HPacket("Chat", "Isso é um teste, favor ignorar", 1, 1))

In [5]:
extension_info = {
    "title": "Extension stuff",
    "description": "g_python test",
    "version": "1.0",
    "author": "sirjonasxx"
}
sys.argv = ['-p', '9092']
ext = Extension(extension_info, sys.argv)   # sys.argv são os argumentos da linha de comando, por exemplo ['-p', '9092'] (porta das extensões do G-Earth)
ext.start()

# Carregar o modelo e o tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

# Função para verificar se uma string está codificada em UTF-8
def is_utf8(data):
    try:
        data.encode('utf-8')
        return True
    except UnicodeError:
        return False

# Função de interceptação e filtragem de mensagens
def all_packets(message):
    packet = message.packet
    packet_str = packet.g_string(ext)
    
    # Extrair a frase da mensagem removendo números e arrays e caracteres extras
    cleaned_message = re.sub(r'[\[\]\d+Æ!)]', '', packet_str).strip()
    
    # Verifica se a mensagem contém a palavra "Rasci" (case-insensitive)
    if re.search(r'rasci', cleaned_message, re.IGNORECASE):
        # Verifica se a mensagem está codificada em UTF-8
        if is_utf8(cleaned_message):
            # Armazena a mensagem capturada
            print(f"Mensagem capturada: {cleaned_message}")
            
            # Gerar resposta usando o modelo de linguagem
            inputs = tokenizer.encode(cleaned_message + tokenizer.eos_token, return_tensors="pt")
            outputs = model.generate(inputs, max_length=25, num_return_sequences=1)
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Enviar a resposta para o servidor
            ext.send_to_server(HPacket("Chat", response, 1, 1))

# Intercepta pacotes indo para o cliente e para o servidor
ext.intercept(Direction.TO_CLIENT, all_packets)

KeyboardInterrupt: 

In [ ]:
extension_info = {
    "title": "Extension stuff",
    "description": "g_python test",
    "version": "1.0",
    "author": "sirjonasxx"
}
sys.argv = ['-p', '9092']
ext = Extension(extension_info, sys.argv)
ext.start()

# Carregar o modelo e o tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

# Função para verificar se uma string está codificada em UTF-8
def is_utf8(data):
    try:
        data.encode('utf-8')
        return True
    except UnicodeError:
        return False

# Variável para armazenar mensagens já respondidas
already_responded = set()

# Função de interceptação e filtragem de mensagens
def all_packets(message):
    packet = message.packet
    packet_str = packet.g_string(ext)
    
    # Extrair a frase da mensagem removendo números e arrays e caracteres extras
    cleaned_message = re.sub(r'[\[\]\d+Æ!)]', '', packet_str).strip()
    
    # Verifica se a mensagem contém a palavra "Rasci" (case-insensitive)
    if re.search(r'kepler', cleaned_message, re.IGNORECASE):
        # Verifica se a mensagem está codificada em UTF-8
        if is_utf8(cleaned_message):
            # Verifica se a mensagem já foi respondida
            if cleaned_message not in already_responded:
                # Armazena a mensagem capturada
                print(f"Mensagem capturada: {cleaned_message}")
                
                # Gerar resposta usando o modelo de linguagem
                inputs = tokenizer.encode(cleaned_message + tokenizer.eos_token, return_tensors="pt")
                outputs = model.generate(inputs, max_length=100, num_return_sequences=1)
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                
                # Enviar a resposta para o servidor
                ext.send_to_server(HPacket("Chat", response, 1, 1))
                
                # Adiciona a mensagem à lista de mensagens já respondidas
                already_responded.add(cleaned_message)

# Intercepta pacotes indo para o cliente e para o servidor
ext.intercept(Direction.TO_CLIENT, all_packets)